<a href="https://colab.research.google.com/github/bigearn23/Coursera_Capsone/blob/main/Segmenting_Clustering_Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PART 1

In [1]:
#import libraries
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
#download data and put into dataframe
import lxml.html as lh
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
#Store the contents of the website
data = lh.fromstring(page.content)
#Store table data
tr_elements = data.xpath('//tr')
col=[]
ind=0
#store each first element and an empty list for each row
for i in tr_elements[0]:
    ind = ind + 1
    name = i.text_content()
    if name.endswith("\n"):
      name = name[:-1]
    col.append((name,[]))
#Since the first row is header, data is stored from the second row
for i in range(1,len(tr_elements)):
    T=tr_elements[i]
    
    #If row is not of size 3, then tr data is not from our table 
    if len(T)!=3:
        break
    #i is the index of our column
    ind=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        temp=t.text_content() 
        #Check if row is empty
        if ind>0:
        #Convert any numerical value to integers
            try:
                temp=int(temp)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[ind][1].append(temp)
        #Increment i for the next column
        ind = ind + 1
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df = df.replace('\n','', regex=True)

df.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [3]:
#Remove 'Not Assigned"
df = df[df.Borough != 'Not assigned']
df['Neighbourhood'] = df.apply(lambda x: x['Borough'] if x['Neighbourhood'] == 'Not assigned' else x['Neighbourhood'], axis=1)
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [4]:
df.shape


(104, 3)

##PART 2

In [5]:
#get long/lat of Toronto
address = 'Toronto, CAN'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6607002, -79.3850889.


In [7]:
 !pip install geocoder   #install if not already available
import geocoder # import geocoder

# define the geocoder function
def get_geocoder(postal_code_from_df):
  lat_lng_coords = None
  while(lat_lng_coords is None):
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df))
    lat_lng_coords = g.latlng
    lat_zip = lat_lng_coords[0]
    long_zip = lat_lng_coords[1]
    return lat_zip, long_zip

#test
get_geocoder('M9B')

     |████████████████████████████████| 102kB 3.1MB/s 


(43.65034000000003, -79.55361999999997)

In [8]:
#could not get geocoder to work, had t o use the csv file
url = 'http://cocl.us/Geospatial_data'
latlong = pd.read_csv(url)
df = df.merge(latlong, left_on='Postal Code', right_on='Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
#Shape
df.shape

(103, 5)

##PART 3

In [18]:
#drop Postal code
#df = df.drop(['Postal Code'], axis=1)
df = df.drop(['Borough'], axis=1)
df.head()

,Neighbourhood,Latitude,Longitude
0,Parkwoods,43.753259,-79.329656
1,Victoria Village,43.725882,-79.315572
2,"Regent Park, Harbourfront",43.654260,-79.360636
3,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
#group rows by Borough
toronto_grouped = df.groupby('Borough')
toronto_grouped

In [19]:
#cluster neighbourhoods
kclusters = 5

toronto_grouped_clustering = df.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 2, 0, 2, 1, 3, 4, 4, 2], dtype=int32)

In [20]:
#add cluster labels to dataframe
df.insert(0, 'Cluster Labels', kmeans.labels_)


df.head() # check the last columns!

,Cluster Labels,Neighbourhood,Latitude,Longitude
0,4,Parkwoods,43.753259,-79.329656
1,4,Victoria Village,43.725882,-79.315572
2,2,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neigh, cluster in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['Cluster Labels']):
    label = folium.Popup(str(neigh) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters